### Predicting customer churn

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [15]:
data = spark.read.csv('customer_churn.csv',inferSchema=True, header=True)

In [16]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [17]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [18]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [21]:
output = assembler.transform(data)

In [22]:
final_data=output.select('features','churn')

In [23]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [24]:
from pyspark.ml.classification import LogisticRegression

In [25]:
lr_churn = LogisticRegression(labelCol='churn')

In [26]:
lr_fit = lr_churn.fit(train_churn)

In [27]:
print(lr_fit.summary.predictions.describe().show())

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                624|                624|
|   mean|0.16826923076923078|0.12980769230769232|
| stddev| 0.3744053193115469| 0.3363613648649312|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+

None


In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [29]:
preds_and_labels = lr_fit.evaluate(test_churn)

In [31]:
preds_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.67003458253027...|[0.99873325179894...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.62797553354771...|[0.99641703615558...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.04972609457998...|[0.99968091251112...|       0.0|
|[28.0,11245.38,0....|    0|[4.11779679589946...|[0.98398045974603...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.99914320496551...|[0.99330145064560...|       0.0|
|[29.0,10203.18,1....|    0|[3.95515235962358...|[0.98120429603728...|       0.0|
|[29.0,12711.15,0....|    0|[5.78603227437554...|[0.99693925739361...|       0.0|
|[30.0,10183.98,1....|    0|[3.00069309408536...|[0.95260542867818...|       0.0|
|[30.0,10960.52,1....|    0|[2.56324087385219...|[0.92845802777571...|       0.0|
|[30.0,12788.37,

In [32]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [33]:
area_under_curve = churn_eval.evaluate(preds_and_labels.predictions)

In [34]:
print(area_under_curve)

0.7783549783549785


In [35]:
final_lr_model = lr_churn.fit(final_data)

In [36]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [38]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [41]:
test_new_customers = assembler.transform(new_customers)

In [42]:
final_results = final_lr_model.transform(test_new_customers)

In [44]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Predict that 4 customers will churn